In [ ]:
include("./trajopt/utils.jl")
include("./trajopt/dynamics.jl")
include("./funlopt/funl_dynamics.jl")
include("./funlopt/funl_utils.jl")
include("./funlopt/funl_synthesis.jl")
include("./funlopt/funl_constraint.jl")
include("./trajopt/scaling.jl")
using Plots
using Random

In [ ]:
using MuJoCo
import MuJoCo: step! as mj_step
using Interpolations
using MatrixEquations

In [ ]:
# load nominal trajectory
using JLD2, FileIO
@load "./data/funl_mani_0710" my_dict
x = my_dict["x"]
u = my_dict["u"]
t = my_dict["t"];
Q = my_dict["Q"];
Y = my_dict["Y"];
Z = my_dict["Z"];
alpha = my_dict["alpha"]
N = size(x,2) - 1
dt = zeros(N)
for i in 1:N
    dt[i] = t[i+1]-t[i]
end

In [ ]:
g = 9.81
dynamics = ThreeDOFManipulatorDynamics(g)
ix = dynamics.ix
iu = dynamics.iu
DLMI = LinearFOH(alpha,ix,iu)
uf = [44.145;19.62;4.905]

model = load_model("./xmls/triplependulum.xml")
data = init_data(model)

In [ ]:
# LQR control for final
ϵ = 1e-6
centred = true
A = mj_zeros(ix, ix)
B = mj_zeros(ix, iu)
data.ctrl .= uf
mjd_transitionFD(model, data, ϵ, centred, A, B, nothing, nothing)
# @show A, B
Qcost = diagm([200, 200, 200, 40, 40 ,40]) # Weights for the state vector
Rcost = diagm([1,1,1])           # Weights for the controls
Scost = zeros(ix, iu)
_, _, Kf, _ = ared(A,B,Rcost,Qcost,Scost)
println(Kf)

In [ ]:
dt_control = model.opt.timestep
sim_time = 3.0
t_ctrl = range(0,sim_time,step=dt_control)

In [ ]:
x_ctrl = zeros(ix,length(t_ctrl))
for idx in 1:ix
    x_fit = LinearInterpolation(t, x[idx,:],extrapolation_bc=Flat());
    x_ctrl[idx,:] .= x_fit(t_ctrl)
end
u_ctrl = zeros(iu,length(t_ctrl))
for idx in 1:iu
    u_fit = LinearInterpolation(t, u[idx,:],extrapolation_bc=Flat());
    u_ctrl[idx,:] .= u_fit(t_ctrl)
end
Q_ctrl = zeros(ix,ix,length(t_ctrl))
Y_ctrl = zeros(iu,ix,length(t_ctrl))
for i in 1:ix
    for j in 1:ix
        Q_fit = LinearInterpolation(t, Q[i,j,:],extrapolation_bc=Flat());
        Q_ctrl[i,j,:] .= Q_fit(t_ctrl)
    end
    for j in 1:iu
        Y_fit = LinearInterpolation(t, Y[j,i,:],extrapolation_bc=Flat());
        Y_ctrl[j,i,:] .= Y_fit(t_ctrl)
    end
end

In [ ]:
xs_list = []
for i in 1:5
    z = randn(ix)
    z = z / norm(z) * 0.5
    push!(xs_list,x[:,1] + sqrt(Q[:,:,1]) * z)
end

In [ ]:
ctrl_trajectories = []
for (idx_s,xs) in enumerate(xs_list)
    reset!(model,data)
    A = mj_zeros(model.nq + model.nv + model.na)
    A[1,:] .= xs
    set_physics_state!(model,data,A')
    forward!(model,data)
    ctrl_states = zeros(6, length(t_ctrl))
    for (idx,t_) in enumerate(t_ctrl)
        @assert(isapprox(t_, data.time))
        ctrl_states[:,idx] = get_physics_state(model, data)
        Q_ = Q_ctrl[:,:,idx]
        Y_ = Y_ctrl[:,:,idx]
        K_ = Y_ * inv(Q_)
        if (t_ <= t[end])
            data.ctrl .= u_ctrl[:,idx] + K_ * (ctrl_states[:,idx] - x_ctrl[:,idx])
        else
            data.ctrl .= uf - Kf * (ctrl_states[:,idx] - zeros(ix))
        end
        mj_step(model, data)
    end
    push!(ctrl_trajectories,ctrl_states)
end

In [ ]:
init_visualiser()
visualise!(model, data, trajectories = [ctrl_trajectories[i] for i in 1:length(xs_list)])

In [ ]:
reset!(model, data)

# Propagate derived quantities
mj_forward(model, data)
# Set joint accelerations to 0
data.qacc .= 0

# Inspect forces from inverse dynamics
mj_inverse(model, data)
println("Required control: ", data.qfrc_inverse)